In [8]:
!pip install pyspark


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from pyspark.sql import SparkSession  # add this import

spark = (
    SparkSession.builder
    .appName("Datacamp Pyspark Tutorial")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "10g")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/09 19:04:21 WARN Utils: Your hostname, TedsMacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.10.3 instead (on interface en0)
25/12/09 19:04:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/09 19:04:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/09 19:04:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [16]:
file_path_job_skills = "../resources/job_skills.csv"
file_path_job_summary = "../resources/job_summary.csv"
file_path_job_postings = "../resources/linkedin_job_postings.csv"

df_job_skills = spark.read.csv(file_path_job_skills, header=True, escape='"', inferSchema=True)
df_job_summary = spark.read.csv(file_path_job_summary, header=True, escape='"', inferSchema=True)
df_job_postings = spark.read.csv(file_path_job_postings, header=True, escape='"', inferSchema=True)


In [18]:
df_job_skills.show(5,0)
df_job_summary.show(5,0)
df_job_postings.show(5,0)

+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|job_link                                                                                                                       |job_skills                                              